In [4]:
import gym
env = gym.make('FrozenLake-v1', is_slippery=False)
observation = env.reset()
for _ in range(100):
    env.render()
    action = env.action_space.sample()
    observation, reward, terminated, done, info = env.step(action)
    print(observation,terminated,done)
    if done or terminated:
        break

(0, {'prob': 1})
0 False False
0 False False
4 False False
5 True False


In [27]:
#dummy Q-learning
import numpy as np

import random
import gym
env = gym.make('FrozenLake-v1', is_slippery=False)
def rargmax(vector):
    m = np.max(vector)
    indices = np.nonzero(vector == m)[0]
    return random.choice(indices)
Q = np.zeros([env.observation_space.n,env.action_space.n])
#learning parameters
num_episodes = 2000
#list of total rewards
rList = []
for i in range(num_episodes):
    #first obversation
    state = env.reset()[0]
    rAll = 0
    done = False
    #Q-learning
    while not done:
        #action with random
        action = rargmax(Q[state,:])
        #get new state and reward
        new_state, reward, terminated, done, info= env.step(action)
        
        #update Q-table
        Q[state,action] = reward + np.max(Q[new_state,:])
        rAll += reward
        state = new_state
    rList.append(rAll)
print("Score over time: " +  str(sum(rList)/num_episodes))
print("Final Q-Table Values")
print('left down right up')
print(Q)


c:\Users\user\anaconda3\envs\nn\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Score over time: 0.938
Final Q-Table Values
left down right up
[[0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 0.]]


In [45]:
#Q-learning with epsilon-greedy, discount factor
import numpy as np
import gym
env = gym.make('FrozenLake-v1', is_slippery=False)
Q = np.zeros([env.observation_space.n,env.action_space.n])
#learning parameters
num_episodes = 3000
r = 0.99

#list of total rewards
rList = []
for i in range(num_episodes):
    #first obversation
    state = env.reset()[0]
    rAll = 0
    done = False
    e = 1./((i//100)+1)
    #Q-learning
    while not done:
        if np.random.rand(1) < e:
            action = env.action_space.sample()
        else:
            action = np.argmax(Q[state,:])
        #get new state and reward
        new_state, reward, terminated, done, info= env.step(action)
        
        #update Q-table
        Q[state,action] = reward + r*np.max(Q[new_state,:])
        rAll += reward
        state = new_state
    rList.append(rAll)
print("Score over time: " +  str(sum(rList)/num_episodes))
print("Final Q-Table Values")
print('left down right up')
print(Q)


Score over time: 0.8586666666666667
Final Q-Table Values
left down right up
[[0.94148015 0.95099005 0.95099005 0.94148015]
 [0.94148015 0.         0.96059601 0.95099005]
 [0.95099005 0.970299   0.95099005 0.96059601]
 [0.96059601 0.         0.95099005 0.95099005]
 [0.95099005 0.96059601 0.         0.94148015]
 [0.         0.         0.         0.        ]
 [0.         0.9801     0.         0.96059601]
 [0.         0.         0.         0.        ]
 [0.96059601 0.         0.970299   0.95099005]
 [0.96059601 0.9801     0.9801     0.        ]
 [0.970299   0.99       0.         0.970299  ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.9801     0.99       0.970299  ]
 [0.9801     0.99       1.         0.9801    ]
 [0.         0.         0.         0.        ]]


: 

In [48]:
import gym
import numpy as np
import random
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import math
%matplotlib inline
env = gym.make('FrozenLake-v1', is_slippery=False)

class Qnet(nn.Module):
    def __init__(self,state_size,action_size):
        super(Qnet,self).__init__()
        self.fc1 = nn.Linear(state_size,state_size)
        self.fc2 = nn.Linear(state_size,action_size)
        self.relu = nn.ReLU()

    def forward(self,x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


c:\Users\user\anaconda3\envs\nn\lib\site-packages\gym\envs\registration.py:555: UserWarning: WARN: The environment FrozenLake-v1 is out of date. You should consider upgrading to version `v3`.
  logger.warn(


In [51]:
#hyperparameters
num_episodes = 1000
learning_rate = 0.1
gamma = 0.99
epsilon_start = 1.0
epsilon_end = 0.01
epsilon_decay = 0.999

#initialize Q-network
state_size = env.observation_space.n
action_size = env.action_space.n
Q = Qnet(state_size,action_size)
optimizer = torch.optim.AdamW(Q.parameters(),lr=0.001)

#epilson-greedy
def select_action(state, epsilon):
    if random.random() < epsilon:
        return env.action_space.sample()  # Explore
    else:
        with torch.no_grad():
            state_tensor = torch.eye(state_size)[state].unsqueeze(0)  # One-hot encode state
            q_values = Q(state_tensor)
            return q_values.argmax().item() 


In [52]:
rewards = []
epsilons = []

for i in range(num_episodes):
    state= env.reset()[0]
    total_reward = 0
    epsilon = epsilon_end + (epsilon_start - epsilon_end) * math.exp(-1. * i / epsilon_decay)
    done = False
    while not done:
        action = select_action(state,epsilon)
        new_state, reward, terminated, done, info = env.step(action)

        q_value = Q(torch.tensor(state, dtype=torch.float32))
        q_hat = Q(torch.tensor(new_state, dtype=torch.float32))
        target = reward + gamma * q_hat.max().item()* (1 - done)
        loss = nn.MSELoss()(q_value[action], torch.tensor(target))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_reward += reward
        state = new_state

        if done:
            break   
    rewards.append(total_reward)
    epsilons.append(epsilon)
# Plotting the rewards
plt.plot(rewards)
plt.title('Rewards over Episodes')
plt.xlabel('Episode')
plt.ylabel('Total Reward')
plt.show()
        



RuntimeError: ArrayRef: invalid index Index = 18446744073709551615; Length = 0